In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import re

# 1.Carregando os dados

In [2]:
dados = pd.read_csv("train_data (1).csv", sep=',', encoding='utf-8', low_memory=False)
dados

,SMS,LABEL
0,BOLETO REFERENTE AS PARCELAS EM ATRASO DO CONSÓRCIO PELO BB.COM VENCIMENTO PARA HOJE Ñ PODE HAVER QUEBRA NO ACORDO. BONATTO ADV 0800 606 3301.,blocked
1,050003DA0202|lcloud-apple-lnc.com/?iphone=VtBqROY .,blocked
2,060804CB5F0303|ps: //bancodobrasil.seguranca.gq,blocked
3,"Atualizacao ainda nao foi executado em sua conta. Evite bloqueios, atualize agora! Acesse: www.avisobbsms.com",blocked
4,0608042BE40303|ps://bancodobrasil.seguranca.cf/,blocked
...,...,...
5995,Host : RB_Bicanga Ip: 170.244.231.14 nao esta respondendo ao ping - 2019-04-19 22:30:23,ok
5996,Host : RB§Bicanga Ip: 170.244.231.14 nao esta respondendo ao ping - 2019-07-05 05:05:17,ok
5997,Host : RB§Bicanga Ip: 170.244.231.14 nao esta respondendo ao ping - 2019-07-15 17:35:17,ok
5998,Host : RB_Bicanga Ip: 170.244.231.14 nao esta respondendo ao ping - 2019-08-02 02:20:20,ok


# 2.Processamento de Linguagem Natural

In [3]:
!pip install nltk

In [4]:
from time import time

In [5]:
ti = time()

In [6]:
import nltk

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gusta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# Lematizador para palavras em Portugues
nltk.download('rslp')

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\gusta\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [9]:
dados

,SMS,LABEL
0,BOLETO REFERENTE AS PARCELAS EM ATRASO DO CONSÓRCIO PELO BB.COM VENCIMENTO PARA HOJE Ñ PODE HAVER QUEBRA NO ACORDO. BONATTO ADV 0800 606 3301.,blocked
1,050003DA0202|lcloud-apple-lnc.com/?iphone=VtBqROY .,blocked
2,060804CB5F0303|ps: //bancodobrasil.seguranca.gq,blocked
3,"Atualizacao ainda nao foi executado em sua conta. Evite bloqueios, atualize agora! Acesse: www.avisobbsms.com",blocked
4,0608042BE40303|ps://bancodobrasil.seguranca.cf/,blocked
...,...,...
5995,Host : RB_Bicanga Ip: 170.244.231.14 nao esta respondendo ao ping - 2019-04-19 22:30:23,ok
5996,Host : RB§Bicanga Ip: 170.244.231.14 nao esta respondendo ao ping - 2019-07-05 05:05:17,ok
5997,Host : RB§Bicanga Ip: 170.244.231.14 nao esta respondendo ao ping - 2019-07-15 17:35:17,ok
5998,Host : RB_Bicanga Ip: 170.244.231.14 nao esta respondendo ao ping - 2019-08-02 02:20:20,ok


## 2.1.Divisão em base treino e base teste para validação do modelo

Vamos separar a base de treino em 2 partes aleatórias, para validar o modelo que será desenvolvido. Para validar o modelo o mais rápido possível, vamos utilizar um simples train_test_split randômico. Se o modelo for aprovado, antes de colocar em produção, o ideal é treiná-lo novamente utilizando a técnica de K-Fold Validation para obter resultados mais robustos, sem dar chance para um possível viés advindo do train_test_split randômico.

In [10]:
import sklearn as sk

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
dados_treino, dados_teste = train_test_split(dados, test_size=0.2, random_state=14, shuffle=True)

In [13]:
dados_treino

,SMS,LABEL
3792,"Faça login no aplicativo OlympTrad?: 131.72.109.159, BR, Embu-guacu",ok
2980,Gruppem SIEM - Outbound cleartext password usage from non guest network detected (Destination IP: 104.44.18.195\r\r\n Destination TCP/UDP Port: 11\r\r\n (SYSTAT)) Veri,ok
2654,"MARCIA, CPF 097.xxx.xxx-04, uma condicao excepcional foi aprovada p/ quitar seu debito junto ao BRADESCO. Ligue agora 08006002934/ wpp 11997193858",ok
3253,"Sr.(a) KARLA,CPF 116.xxx.xxx-76, seu debito junto ao BRADESCO desde 15/09/2006 pode ser quitado hoje por R$600. Ligue agora 08006002934 /whatsapp 11997193858",ok
1446,BB INFORMA:VALIDE SUA SENHA E EVITE TRANSTORNO. ACESSE: www.Bbrasildesbloqueio.com/?GFQKE15S,blocked
...,...,...
3833,Roteador RadioScan Externo (132.255.248.169)\r\nDesligado as Jan 02 2019 10:20 PM (GMT -2:00)\r\nErro: ping: request timeout,ok
2454,"Estamos tentando contato com ALINE. Assunto de extrema importancia, CPF: 073-xxx.xxx-09 . Entre em contato pelo telefone: 08007213569 ou Whats: (11)976509508",ok
1292,BB INFORMA:VALIDE SUA SENHA E EVITE TRANSTORNO. ACESSE: www.Bbrasildesbloqueio.com/?F74F5SEG,blocked
5464,"Faça login no aplicativo OlympTrad?: 168.121.182.0, BR, Mesquita",ok


In [14]:
tamanho_base_treino = len(dados_treino)
tamanho_base_treino_ok = len(dados_treino[(dados_treino['LABEL'])=='ok'])
tamanho_base_treino_blocked = len(dados_treino[(dados_treino['LABEL'])=='blocked'])

porcentagem_ok = (tamanho_base_treino_ok * 100) / tamanho_base_treino
porcentagem_blocked = 100 - porcentagem_ok

print("A base treino tem {:.4}% de mensagens com classificação OK".format(porcentagem_ok))
print("A base treino tem {:.4}% de mensagens com classificação BLOCKED".format(porcentagem_blocked))

A base treino tem 75.27% de mensagens com classificação OK
A base treino tem 24.73% de mensagens com classificação BLOCKED


In [15]:
dados_teste

,SMS,LABEL
1848,"Sr.(a) JAIRO,CPF 018.xxx.xxx-83, seu debito junto ao BRADESCO desde 25/11/2012 pode ser quitado hoje por R$1600. Ligue agora 08006002934 /whatsapp 11997193858",ok
532,Banco Santander.\r\r\n\r\r\nPrezado(a) Cliente evite o bloqueio de seu cartao e senhas. Refaca a Operacao. Acesse: http://lu.x-sms-santander.com/?5511981870490,blocked
5903,"Faça login no aplicativo OlympTrad?: 170.233.180.207, BR, Turvo",ok
1302,BB INFORMA:VALIDE SUA SENHA E EVITE TRANSTORNO. ACESSE: www.Bbrasildesbloqueio.com/?Z0MY2DD8,blocked
787,BB INFORMA: SUA SENHA J EXPIROU ACESSE E VALIDE AGORA: www.Bbrasildesbloqueio.com/?US16P1GQ,blocked
...,...,...
2501,"CARLOS, CPF 077.xxx.xxx-35, uma condicao excepcional foi aprovada p/ quitar seu debito junto ao BRADESCO. Ligue agora 08006002934/ wpp 11997193858",ok
5319,-- ping (inativo) -- PING.NET_(Bruno) -- as 08:04:10. IP: 167.249.95.10,ok
2505,"CASA,CNPJ 07799.xxx/xxxx-00,proposta p/ quitar seu debito referente a RENEGOCIACAO DE DIVIDA por R$15.51 valida ate 08/03. Ligue 08006002934 /whats 11997193858",ok
5906,"Faça login no aplicativo OlympTrad?: 170.233.206.46, BR, Areia Branca",ok


In [16]:
tamanho_base_teste = len(dados_teste)
tamanho_base_teste_ok = len(dados_teste[(dados_teste['LABEL'])=='ok'])
tamanho_base_teste_blocked = len(dados_teste[(dados_teste['LABEL'])=='blocked'])

porcentagem_ok = (tamanho_base_teste_ok * 100) / tamanho_base_teste
porcentagem_blocked = 100 - porcentagem_ok

print("A base teste tem {:.4}% de mensagens com classificação OK".format(porcentagem_ok))
print("A base teste tem {:.4}% de mensagens com classificação BLOCKED".format(porcentagem_blocked))

A base teste tem 73.92% de mensagens com classificação OK
A base teste tem 26.08% de mensagens com classificação BLOCKED


In [17]:
dados_treino.reset_index(drop=True, inplace=True)
dados_treino

,SMS,LABEL
0,"Faça login no aplicativo OlympTrad?: 131.72.109.159, BR, Embu-guacu",ok
1,Gruppem SIEM - Outbound cleartext password usage from non guest network detected (Destination IP: 104.44.18.195\r\r\n Destination TCP/UDP Port: 11\r\r\n (SYSTAT)) Veri,ok
2,"MARCIA, CPF 097.xxx.xxx-04, uma condicao excepcional foi aprovada p/ quitar seu debito junto ao BRADESCO. Ligue agora 08006002934/ wpp 11997193858",ok
3,"Sr.(a) KARLA,CPF 116.xxx.xxx-76, seu debito junto ao BRADESCO desde 15/09/2006 pode ser quitado hoje por R$600. Ligue agora 08006002934 /whatsapp 11997193858",ok
4,BB INFORMA:VALIDE SUA SENHA E EVITE TRANSTORNO. ACESSE: www.Bbrasildesbloqueio.com/?GFQKE15S,blocked
...,...,...
4795,Roteador RadioScan Externo (132.255.248.169)\r\nDesligado as Jan 02 2019 10:20 PM (GMT -2:00)\r\nErro: ping: request timeout,ok
4796,"Estamos tentando contato com ALINE. Assunto de extrema importancia, CPF: 073-xxx.xxx-09 . Entre em contato pelo telefone: 08007213569 ou Whats: (11)976509508",ok
4797,BB INFORMA:VALIDE SUA SENHA E EVITE TRANSTORNO. ACESSE: www.Bbrasildesbloqueio.com/?F74F5SEG,blocked
4798,"Faça login no aplicativo OlympTrad?: 168.121.182.0, BR, Mesquita",ok


In [18]:
dados_teste.reset_index(drop=True, inplace=True)
dados_teste

,SMS,LABEL
0,"Sr.(a) JAIRO,CPF 018.xxx.xxx-83, seu debito junto ao BRADESCO desde 25/11/2012 pode ser quitado hoje por R$1600. Ligue agora 08006002934 /whatsapp 11997193858",ok
1,Banco Santander.\r\r\n\r\r\nPrezado(a) Cliente evite o bloqueio de seu cartao e senhas. Refaca a Operacao. Acesse: http://lu.x-sms-santander.com/?5511981870490,blocked
2,"Faça login no aplicativo OlympTrad?: 170.233.180.207, BR, Turvo",ok
3,BB INFORMA:VALIDE SUA SENHA E EVITE TRANSTORNO. ACESSE: www.Bbrasildesbloqueio.com/?Z0MY2DD8,blocked
4,BB INFORMA: SUA SENHA J EXPIROU ACESSE E VALIDE AGORA: www.Bbrasildesbloqueio.com/?US16P1GQ,blocked
...,...,...
1195,"CARLOS, CPF 077.xxx.xxx-35, uma condicao excepcional foi aprovada p/ quitar seu debito junto ao BRADESCO. Ligue agora 08006002934/ wpp 11997193858",ok
1196,-- ping (inativo) -- PING.NET_(Bruno) -- as 08:04:10. IP: 167.249.95.10,ok
1197,"CASA,CNPJ 07799.xxx/xxxx-00,proposta p/ quitar seu debito referente a RENEGOCIACAO DE DIVIDA por R$15.51 valida ate 08/03. Ligue 08006002934 /whats 11997193858",ok
1198,"Faça login no aplicativo OlympTrad?: 170.233.206.46, BR, Areia Branca",ok


## 2.2.Estruturação dos dados

Para facilitar todo o processo de criação do algoritmo de NLP, vamos estruturar os dados, transformando o dataframe numa lista de tuplas, com cada tupla contendo a mensagem e a classificação. Vamos criar uma função para isso, pois pode ser útil no decorrer do projeto.

In [19]:
def carregar_dados(dados):
    base = []
    for i in range(len(dados)):
        base.append((dados['SMS'].loc[i], dados['LABEL'].loc[i]))
    return base

In [20]:
base_treino = carregar_dados(dados_treino)

In [21]:
base_treino[0]

('Faça login no aplicativo OlympTrad?: 131.72.109.159, BR, Embu-guacu', 'ok')

In [22]:
base_teste = carregar_dados(dados_teste)

In [23]:
base_teste[0]

('Sr.(a) JAIRO,CPF 018.xxx.xxx-83, seu debito junto ao BRADESCO desde 25/11/2012 pode ser quitado hoje por R$1600. Ligue agora 08006002934 /whatsapp 11997193858',
 'ok')

In [24]:
print("Tamanho base de treino: ", len(base_treino))
print("Tamanho base de teste: ", len(base_teste))

Tamanho base de treino:  4800
Tamanho base de teste:  1200


## 2.3.Pré-processamento

### 2.3.1.Remoção de pontução

In [25]:
import string

In [26]:
def remover_pontuacao(base):
    """Essa função remove as pontuações da base.
    Args:
        base: contém as mensagens no formato (SMS,LABEL).
    Returns:
        base_dados: É uma lista de tuplas.
    """
    lista_final = []
    for (sms, label) in base:
        sem_pontuacao = []
        # Para cada palavra na mensagem        
        for p in sms:
            # Verifica se não é uma pontuação
            if p not in string.punctuation:
                sem_pontuacao.append(p)
        # Refaz a frase
        aux = ''.join(sem_pontuacao)
        # Salva na lista final no formato (sms,label)
        lista_final.append((aux.lower(), label))
    # Retorna todo o conjunto sem as pontuações
    return (lista_final)

In [27]:
print(base_treino[0])

('Faça login no aplicativo OlympTrad?: 131.72.109.159, BR, Embu-guacu', 'ok')


In [28]:
print(remover_pontuacao([base_treino[0]]))

[('faça login no aplicativo olymptrad 13172109159 br embuguacu', 'ok')]


In [29]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [30]:
frases_sem_pontuacao = remover_pontuacao(base_treino)

In [31]:
print(frases_sem_pontuacao[0])

('faça login no aplicativo olymptrad 13172109159 br embuguacu', 'ok')


### 2.3.2.Remoção de Stop Words

As stopwords são palavras que não possuem significado para o desempenho dos algoritmos de classificação de texto. Por exemplo: 'é', 'muito', 'o', 'por', entre outros.

A permanência delas na base de dados pode provocar maior lentidão no processamento dos dados, sem utilidade para o contexto em que estamos trabalhando.

A função ```remover_stopwords``` funciona da seguinte forma:

- O parâmetro "frases_sem_pontuacao" representa toda a base de dados já tratada.
- Essa base de dados contém um par de elementos sendo o primeiro a frase e o segundo a classe (ok, blocked).
- A função percorre toda a base de dados, linha a linha, verificando quais frases possuem as stopwords definidas na biblioteca NLTK. Uma vez identificadas, elas são removidas da frase. 
- O conjunto final é uma estrutura contendo a frase (sem as stopwords) e a classe (ok, blocked).

In [32]:
# Carrega as stopwords definidas na biblioteca para o idioma Português
stopwordsnltk = nltk.corpus.stopwords.words('portuguese')

# Visualiza algumas
print(stopwordsnltk[:10])

['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um']


In [33]:
def remover_stopwords(frases_sem_pontuacao):
    """Essa função remove as stopwords da base.
    Args:
        frases_sem_pontuacao: É uma lista de tuplas.
    Returns:
        frases_final: É uma lista de tuplas (frase, classe).
    """
    stopwordsnltk = nltk.corpus.stopwords.words('portuguese')
    frases_final = []
    for (frase, classe) in frases_sem_pontuacao:
        sem_stop = []
        for palavra in frase.split():
            if palavra not in stopwordsnltk:
                sem_stop.append(palavra)  
        frases_final.append((sem_stop, classe))
    return frases_final

In [34]:
frases_sem_stopwords = remover_stopwords(frases_sem_pontuacao)

In [35]:
print(frases_sem_stopwords[0])

(['faça', 'login', 'aplicativo', 'olymptrad', '13172109159', 'br', 'embuguacu'], 'ok')


### 2.3.3.Remoção do Radical das palavras (stemming)

Stemming é uma técnica utilizada para reduzir a dimensionalidade dos dados na etapa de pré-processamento. É baseada na redução de palavras em seu morfema, de acordo com as regras do idioma que o algoritmo será executado. 

Por exemplo, em português a palavra “casa” possui o morfema “cas” e suas variações: casinhas, casebre, casona.

A função ```aplicar_stemmer``` funciona da seguinte forma:

- Utiliza-se uma ferramenta da biblioteca NLTK para realizar essa técnica. Para isso, acessamos o pacote ```stem``` para realizar essa tarefa.
- ```nltk.stem.RSLPStemmer()``` indica que será utilizado a lingua portguesa.
- O parâmetro ```frases_sem_stopwords``` da função ```aplicar_stemmer``` representa a base de dados sem as stopwords que foram removidas anteriormente.
- A função percorre toda a base de dados aplicando o método ```stemmer.stem(palavra)``` em cada palavra de cada frase, cuja finalidade é deixar apenas o radical de cada palavra.

Uma desvantagem da aplicação do algoritmo stemmer é quando duas palavras com sentidos diferentes possuem o mesmo radical, como por exemplo as palavras ```novamente``` e ```novo``` que possuem o radical ```nov``` dessa forma, na etapa de aprendizado de máquina o algoritmo pode perder algumas nuances.

In [36]:
def aplicar_stemmer(frases_sem_stopwords):
    """Função que reduz a palavra ao seu radical
    Args:
        frases_sem_stopwords: lista de tuplas.
    Returns:
        frases_stemming: lista de tuplas.
    """
    stemmer = nltk.stem.RSLPStemmer()
    frases_stemming =[]
    for (frase, classe) in frases_sem_stopwords:
        com_stemming = []
        # Para cada palavra na frase, aplicar o stemmer e salvar
        for palavra in frase:
            com_stemming.append(str(stemmer.stem(palavra)))
        frases_stemming.append((com_stemming, classe))    
    # Retornar todo o conjunto com o stemming aplicado
    return frases_stemming

In [37]:
frases_com_stemmer = aplicar_stemmer(frases_sem_stopwords)

In [38]:
print(frases_com_stemmer[0])

(['faç', 'login', 'aplic', 'olymptrad', '13172109159', 'br', 'embuguacu'], 'ok')


### 2.3.4.Lista de todas as palavras

A função ```extrair_palavras``` irá gerar uma nova lista com todas as palavras que já foram pré-processadas anteriormente porém sem a sua classificação (ok, blocked) associada.

Funciona da seguinte forma:
- O parâmetro da função representa a lista gerada pela função aplicar_stemmer, que é a ```frases_com_stemmer```.
- Ela percorre toda a base e insere em uma lista com todas as palavras da base de dados, mas sem sua classificação associada.

In [39]:
def extrair_palavras(frases_com_stemmer):
    """Função que unifica todas as palavras do conjunto de dados em uma única lista.
    Args:
        frases_com_stemmer: Frases com o Stemmer já aplicados.
    Returns:
        todas_palavras: lista com todas as palavras.
    """
    todas_palavras = []
    for (palavras, classe) in frases_com_stemmer:
        todas_palavras.extend(palavras)
    return todas_palavras

In [40]:
palavras_sem_classe = extrair_palavras(frases_com_stemmer)

In [41]:
palavras_sem_classe[:10]

['faç',
 'login',
 'aplic',
 'olymptrad',
 '13172109159',
 'br',
 'embuguacu',
 'grupp',
 'si',
 'outbound']

### 2.3.5.Medição de frequência de palavras únicas

Com a função ```aplicar_frequencia``` iremos remover os radicais repetidos da base para otimizar o processamento dos dados utilizando o recurso do nltk ```FreqDist```.

A classe ```FreqDist``` unifica todas as palavras repetidas gerando um dicionário do tipo ```chave,valor``` dentro de uma lista, sendo a chave o radical e o valor a frequencia com que ele se repete. Ex: ```('am', 4)```. Nesse exemplo o radical ```am``` apareceu na base de dados 4 vezes.

In [42]:
def aplicar_frequencia(palavras_sem_classe):
    """Função que aplica a frequencia das palavras
    Args:
        palavras_sem_classe: palavras sem a classificação.
    Returns:
        palavras: FreqDist
    """
    palavras = nltk.FreqDist(palavras_sem_classe)
    return palavras

In [43]:
frequencia_palavras = aplicar_frequencia(palavras_sem_classe)

In [44]:
print(frequencia_palavras.most_common(50))

[('login', 1317), ('olymptrad', 1289), ('br', 1132), ('aplic', 1113), ('acess', 989), ('faç', 886), ('lig', 845), ('senh', 799), ('cpf', 795), ('debit', 761), ('bb', 707), ('evit', 630), ('quit', 609), ('08006002934', 609), ('11997193858', 609), ('what', 593), ('contat', 532), ('transtorn', 505), ('informavalid', 504), ('agor', 495), ('ate', 484), ('p', 478), ('ip', 466), ('junt', 408), ('brad', 407), ('refer', 348), ('val', 348), ('0803', 337), ('hoj', 303), ('telefon', 278), ('pod', 275), ('tent', 271), ('r', 262), ('inform', 260), ('whatsapp', 256), ('assunt', 256), ('extrem', 256), ('importanc', 256), ('11976509508', 256), ('lanc', 249), ('propost', 249), ('ping', 213), ('nov', 197), ('voc', 196), ('j', 181), ('expir', 181), ('valid', 181), ('proprietari', 172), ('div', 167), ('renegociaca', 165)]


### 2.3.6.Extração das palavras únicas

Vamos criar uma estrutura apenas com as palavras únicas da frequência gerada anteriormente, utilizando a função ```extrair_palavras_unicas```.

In [45]:
def extrair_palavras_unicas(frequencia_palavras):
    """Função que retorna as palavras únicas
    Args:
        frequencia_palavras: dicionário com a frequencia das palavras.
    Returns:
        freq: palavras únicas.
    """
    freq = frequencia_palavras.keys()
    return freq

In [46]:
palavras_sem_repeticao = extrair_palavras_unicas(frequencia_palavras)

In [47]:
print(list(palavras_sem_repeticao)[:5])

['faç', 'login', 'aplic', 'olymptrad', '13172109159']


### 2.3.7.Extração de palavras de cada frase

O objetivo da função ```criar_caracteristicas``` é auxiliar a caracterização das frases a serem utilizadas no algoritmo Naive Bayes.

O método ```nltk.classify.apply_features``` realiza essa caracterização através do mapeamento de cada frase na função ```criar_caracteristicas```. O resultado desse mapeamento é um dicionário para cada frase onde as palavras que pertencem a respectiva frase sejam ```True```. Todas as outras palavras da base que não pertencem a frase serão definidas como ```False```.

O método ```nltk.classify.apply_features``` exige dois parâmetros, sendo o primeiro uma função que irá extrair as caracteristicas e o segundo é o conjunto de dados onde será aplicado essa caracterização.

Essa etapa é necessária para a preparação da base de dados para o algoritmo de aprendizagem de máquina Naive Bayes. É o resultado dessa função que irá ser passada como parâmetro para criar o classificador.

A função ```criar_caracteristicas``` recebe a lista com as palavras com o stemming e cria uma estrutura onde apenas as palavras que estão nessa lista será marcada como ```True```, todas as outras serão marcadas como ```False```.

In [48]:
def criar_caracteristicas(documento):
    """Função que cria as características do documento, verificando se a palavra existe ou não no documento.
    Args:
        documento: lista com todas as palavras
    Returns:
        caracteristicas: dicionário com as características.
    """
    global palavras_sem_repeticao
    doc = set(documento)
    caracteristicas = {}
    # Para cada palavra
    for palavra in palavras_sem_repeticao:
        # Se a palavra existir no documento é atribuido True, caso contrário False.
        caracteristicas[palavra] = (palavra in doc)    
    # Dicionário com as caracteristicas da palavra
    return caracteristicas

In [49]:
# Para testar a função acima, serão utilizadas duas frases com o stemmer já aplicado.
teste_caracteristica = frases_com_stemmer[0:2]

In [50]:
print(teste_caracteristica)

[(['faç', 'login', 'aplic', 'olymptrad', '13172109159', 'br', 'embuguacu'], 'ok'), (['grupp', 'si', 'outbound', 'cleartext', 'password', 'usag', 'from', 'non', 'guest', 'network', 'detected', 'destination', 'ip', '1044418195', 'destination', 'tcpudp', 'port', '11', 'systat', 'ver'], 'ok')]


In [51]:
print(extrair_palavras(teste_caracteristica))

['faç', 'login', 'aplic', 'olymptrad', '13172109159', 'br', 'embuguacu', 'grupp', 'si', 'outbound', 'cleartext', 'password', 'usag', 'from', 'non', 'guest', 'network', 'detected', 'destination', 'ip', '1044418195', 'destination', 'tcpudp', 'port', '11', 'systat', 'ver']


A execução ```nltk.classify.apply_features``` irá gerar uma lista, onde cada elemento dessa lista é uma tupla com dois elementos, sendo o primeiro o dicionário gerado pela função ```criar_caracteristicas``` e o segundo elemento com a classificação (ok, blocked).

In [52]:
frases_teste_final = nltk.classify.apply_features(criar_caracteristicas, teste_caracteristica)

In [53]:
# Visualizar 1 elemento classificado
print(frases_teste_final[0])

({'faç': True, 'login': True, 'aplic': True, 'olymptrad': True, '13172109159': True, 'br': True, 'embuguacu': True, 'grupp': False, 'si': False, 'outbound': False, 'cleartext': False, 'password': False, 'usag': False, 'from': False, 'non': False, 'guest': False, 'network': False, 'detected': False, 'destination': False, 'ip': False, '1044418195': False, 'tcpudp': False, 'port': False, '11': False, 'systat': False, 'ver': False, 'marc': False, 'cpf': False, '097xxxxxx04': False, 'condica': False, 'excepc': False, 'aprov': False, 'p': False, 'quit': False, 'debit': False, 'junt': False, 'brad': False, 'lig': False, 'agor': False, '08006002934': False, 'wpp': False, '11997193858': False, 'sra': False, 'karlacpf': False, '116xxxxxx76': False, 'desd': False, '15092006': False, 'pod': False, 'ser': False, 'hoj': False, 'r600': False, 'whatsapp': False, 'bb': False, 'informavalid': False, 'senh': False, 'evit': False, 'transtorn': False, 'acess': False, 'wwwbbrasildesbloqueiocomgfqke15': Fals

Para facilitar a reutilização de todo o código criado anteriormente, iremos criar uma função para estruturar qualquer texto que desejamos classificar.

In [54]:
def estruturar_dados(base):
    """Dada uma base de dados, é realizada toda a estruturação das bases.
    Args:
        base: contém todas as mensagens no formato (SMS,LABEL).
    Returns:
        base_final: conjunto de dados estruturados.
    """
    global palavras_sem_repeticao
    # Aplicar as funções previamente definidas
    frases_sem_pontuacao = remover_pontuacao(base)
    frases_sem_stopwords = remover_stopwords(frases_sem_pontuacao)
    frases_com_stemmer = aplicar_stemmer(frases_sem_stopwords)
    palavras_sem_classe = extrair_palavras(frases_com_stemmer)
    frequencia_palavras = aplicar_frequencia(palavras_sem_classe)
    palavras_sem_repeticao = extrair_palavras_unicas(frequencia_palavras)
    base_final = nltk.classify.apply_features(criar_caracteristicas, frases_com_stemmer)
    # Retornar os dados estruturados para serem utilizados pela função NLTK.
    return (base_final)

# 3.Treino do algoritmo Naive Bayes

Nesta etapa, será realizado o treino do algoritmo Naive Bayes, que irá gerar um modelo a ser utilizado na classificação de novas frases em ok ou blocked.

## 3.1.Classificação do texto

O algoritmo de Naive Bayes realiza a análise estatística e monta uma tabela de probabilidade. Após isso, é criada a classificação dos registros.

O método ```train``` da classe ```NaiveBayesClassifier``` recebe como parâmetro a ```base_treino``` já estruturada (```base_final```) e realiza a etapa de construção da tabela de probabilidades. O método ```show_most_informative_features``` retorna os atributos (palavras) mais significativos.

Por exemplo: 

- ```dia = True ok : blocke = 2.3:1.0``` - Neste exemplo de saída a probabilidade de a frase ser classificada como ```ok``` quando a palavra "dia" estiver presente na frase (```True```) é 2.3 vezes maior do que blocked.

- ```am = False blocke : ok = 1.6:1.0``` - Já neste exemplo, a probabilidade de a frase ser classificada como ```blocked``` quando a palavra ```am``` **não** estiver presente na frase (```False```) é 1.6 vezes maior do que ok.

In [55]:
base_treino

[('Faça login no aplicativo OlympTrad?: 131.72.109.159, BR, Embu-guacu', 'ok'),
 ('Gruppem SIEM - Outbound cleartext password usage from non guest network detected (Destination IP: 104.44.18.195\r\r\n  Destination TCP/UDP Port: 11\r\r\n (SYSTAT)) Veri',
  'ok'),
 ('MARCIA, CPF 097.xxx.xxx-04, uma condicao excepcional foi aprovada p/ quitar seu debito junto ao BRADESCO. Ligue agora 08006002934/ wpp 11997193858',
  'ok'),
 ('Sr.(a) KARLA,CPF 116.xxx.xxx-76, seu debito junto ao BRADESCO desde 15/09/2006 pode ser quitado hoje por R$600. Ligue agora 08006002934 /whatsapp 11997193858',
  'ok'),
 ('BB INFORMA:VALIDE SUA SENHA E EVITE TRANSTORNO. ACESSE: www.Bbrasildesbloqueio.com/?GFQKE15S',
  'blocked'),
 ('bitnuvem.com: Identificamos acesso do IP 138.204.25.45 na sua conta. Nao conhece esse IP? Acesse sua conta e atualize sua senha imediatamente.',
  'ok'),
 ('Israel, Voce tem uma nova foto http://fotosms.xyz/c/d?i=6E3NEpQxp4',
  'blocked'),
 ('Estamos tentando contato com ANA. Assunto de e

In [56]:
base_final = estruturar_dados(base_treino)

In [57]:
base_final[0]

({'faç': True,
  'login': True,
  'aplic': True,
  'olymptrad': True,
  '13172109159': True,
  'br': True,
  'embuguacu': True,
  'grupp': False,
  'si': False,
  'outbound': False,
  'cleartext': False,
  'password': False,
  'usag': False,
  'from': False,
  'non': False,
  'guest': False,
  'network': False,
  'detected': False,
  'destination': False,
  'ip': False,
  '1044418195': False,
  'tcpudp': False,
  'port': False,
  '11': False,
  'systat': False,
  'ver': False,
  'marc': False,
  'cpf': False,
  '097xxxxxx04': False,
  'condica': False,
  'excepc': False,
  'aprov': False,
  'p': False,
  'quit': False,
  'debit': False,
  'junt': False,
  'brad': False,
  'lig': False,
  'agor': False,
  '08006002934': False,
  'wpp': False,
  '11997193858': False,
  'sra': False,
  'karlacpf': False,
  '116xxxxxx76': False,
  'desd': False,
  '15092006': False,
  'pod': False,
  'ser': False,
  'hoj': False,
  'r600': False,
  'whatsapp': False,
  'bb': False,
  'informavalid': False,

In [58]:
%%time
# Cria o classificador (tabela de probabilidade) com base no conjunto de treinamento
classificador = nltk.NaiveBayesClassifier.train(base_final)

Wall time: 54 s


In [59]:
# Retorna as classes da base de dados (ok, blocked)
print(classificador.labels())

['ok', 'blocked']


In [60]:
# Retorna os 10 atributos mais significativos
print(classificador.show_most_informative_features(10))

Most Informative Features
                      bb = True           blocke : ok     =   1432.8 : 1.0
                      br = True               ok : blocke =    247.7 : 1.0
                       j = True           blocke : ok     =    218.4 : 1.0
                   debit = True               ok : blocke =    165.1 : 1.0
                 prezado = True           blocke : ok     =    157.2 : 1.0
                   aplic = True               ok : blocke =    146.1 : 1.0
                   local = True           blocke : ok     =    132.8 : 1.0
                   login = True               ok : blocke =    122.5 : 1.0
                     lig = True               ok : blocke =    110.9 : 1.0
                    banc = True           blocke : ok     =     80.9 : 1.0
None


### 3.2.Testando o classificador

Para testar o classificador, iremos utilizar uma frase para verificar a classificação realizada, uma vez que a base de dados já foi pré-processada e treinada com a base de dados para treino.

Para que a nova frase seja classificada temos que realizar toda a fase de pré-processamento. Como criamos a função ```estruturar_dados```, podemos simplesmente utilizá-la.

In [61]:
base_teste[0]

('Sr.(a) JAIRO,CPF 018.xxx.xxx-83, seu debito junto ao BRADESCO desde 25/11/2012 pode ser quitado hoje por R$1600. Ligue agora 08006002934 /whatsapp 11997193858',
 'ok')

In [62]:
frase_teste = estruturar_dados([base_teste[0]])
print(frase_teste)

[({'sra': True, 'jairocpf': True, '018xxxxxx83': True, 'debit': True, 'junt': True, 'brad': True, 'desd': True, '25112012': True, 'pod': True, 'ser': True, 'quit': True, 'hoj': True, 'r1600': True, 'lig': True, 'agor': True, '08006002934': True, 'whatsapp': True, '11997193858': True}, 'ok')]


In [63]:
print(frase_teste[0][0])
caracteristica_teste = frase_teste[0][0]

{'sra': True, 'jairocpf': True, '018xxxxxx83': True, 'debit': True, 'junt': True, 'brad': True, 'desd': True, '25112012': True, 'pod': True, 'ser': True, 'quit': True, 'hoj': True, 'r1600': True, 'lig': True, 'agor': True, '08006002934': True, 'whatsapp': True, '11997193858': True}


In [64]:
# Realizar a classificação
print(classificador.classify(caracteristica_teste))

ok


Quando chamamos o método ```classificador.classify``` com o parâmetro ```frase_teste```, o algoritmo classifica a frase como rótulo ```ok```.

Para visualizar a distribuição de probabilidade utiliza-se o método ```prob_classify``` que mostra a porcentagem para cada uma das classes.

In [65]:
# Retorna a classe e o valor da distribuição de probabilidade
distribuicao = classificador.prob_classify(caracteristica_teste)

# Para cada classe, verifica-se a probabilidade
for classe in distribuicao.samples():
    print('%s: %f' % (classe, distribuicao.prob(classe)))

ok: 1.000000
blocked: 0.000000


# 4.Teste do algoritmo Naive Bayes

Na etapa de teste, utiliza-se outro conjunto de dados, com o objetivo de testar o algoritmo de aprendizado de máquina com novas frases. Para tal, a base de dados deve conter frases diferentes da base de treinamento e sem a informação de sua classificação (ok, blocked).

O algoritmo é executado novamente e o método ```classify.accuracy```  mostra a proximidade entre o percentual obtido experimentalmente e o valor verdadeiro da classificação das frases.

Passamos como parâmetro o classificador, que nada mais é do que uma tabela de probabilidade que o Naive Bayes gera, e a base de dados para teste.

In [66]:
frases_teste = estruturar_dados(base_teste)

In [67]:
print(frases_teste[0])

({'sra': True, 'jairocpf': True, '018xxxxxx83': True, 'debit': True, 'junt': True, 'brad': True, 'desd': True, '25112012': True, 'pod': True, 'ser': True, 'quit': True, 'hoj': True, 'r1600': True, 'lig': True, 'agor': True, '08006002934': True, 'whatsapp': True, '11997193858': True, 'banc': False, 'santand': False, 'prezado': False, 'client': False, 'evit': False, 'bloquei': False, 'carta': False, 'senh': False, 'refac': False, 'operaca': False, 'acess': False, 'httpluxsmssantandercom5511981870490': False, 'faç': False, 'login': False, 'aplic': False, 'olymptrad': False, '170233180207': False, 'br': False, 'turv': False, 'bb': False, 'informavalid': False, 'transtorn': False, 'wwwbbrasildesbloqueiocomz0my2dd8': False, 'inform': False, 'j': False, 'expir': False, 'valid': False, 'wwwbbrasildesbloqueiocomus16p1gq': False, 'wwwbbrasildesbloqueiocomlgqrka95': False, 'sent': False, 'from': False, 'your': False, 'twili': False, 'trial': False, 'account': False, 'entr': False, 'cont': False, 

O método ```classify.accuracy``` funciona da seguinte forma: ele submete todos os registros da base de teste ao classificador e o classificador gera uma classificação para cada um dos registros. Após isso, realiza uma comparação entre a classificação gerada e a classifcação que já tinha sido realizada na base de dados, e devolve a taxa de acerto.

In [68]:
acurácia = nltk.classify.accuracy(classificador, frases_teste)
acurácia_percentual = acurácia*100

print("A acurácia da classificação da base teste é de {:.4}%".format(acurácia_percentual))

A acurácia da classificação da base teste é de 98.42%


In [69]:
res = {'ok' : 0, 'blocked' : 0}
total = 0
for (texto, classe) in base_teste:
    if classe == 'ok':
        res[classe] += 1
    elif classe == 'blocked':
        res[classe] += 1
    total += 1
print(res)

{'ok': 887, 'blocked': 313}


In [70]:
887 / (887 + 313)

0.7391666666666666

Esse resultado possibilita realizar algumas análises, tais como:

1. **Análise de cenário**: O percentual de acerto do algoritmo é bom ou ruim? R: É muito bom! Corre o risco de overfitting.
2. **Análise do número de classes**: A probabilidade mínima aceitável para o algoritmo ser melhor do que usar a aleatoriedade é que a acurácia seja maior que 50%, ou seja, dividir 100% pela quantidade de classes.
3. **ZeroRules**: Nessa análise, estamos comparando o resultado obtido pelo sistema, com o método de classificar uma frase de acordo com a classe que possui maior quantidade de frases na base de dados de treino e teste. Por exemplo, dividimos a classe com maior número de registros pelo total de registros na base de dados (```887/1200 = 73,92%```). Desta forma, conclui-se que o sistema apresenta mais acertos do que classificar todas as novas frases nessa classe.

## 5.Visualização de erros do algoritmo

É possível visualizar a classe já pré-classificada, a classe que o algoritmo classificou e a frase vinculada ao erro gerado.

Por exemplo: ```ok blocked {'sra': False,'jairocpf': False,'018xxxxxx83': False,'debit': False,'junt': False, ... ,'sent': True,'from': True,'your': True,'twili': True,...}```. Essa saída nos diz qual a classe correta, ou seja, aquela que está na base de dados para teste é ```ok```. O algoritmo classificou como ```blocked``` e a frase vinculada à classificação possui os radicais ```sent``` e ```from```.

Para identificar corretamente os acertos e erros podemos:

In [71]:
erros =[]
for (frase, classe) in frases_teste:
    resultado = classificador.classify(frase)
    
    if resultado != classe:
        erros.append((classe, resultado, frase))

In [72]:
erros[0]

('ok',
 'blocked',
 {'sra': False,
  'jairocpf': False,
  '018xxxxxx83': False,
  'debit': False,
  'junt': False,
  'brad': False,
  'desd': False,
  '25112012': False,
  'pod': False,
  'ser': False,
  'quit': False,
  'hoj': False,
  'r1600': False,
  'lig': False,
  'agor': False,
  '08006002934': False,
  'whatsapp': False,
  '11997193858': False,
  'banc': False,
  'santand': False,
  'prezado': False,
  'client': False,
  'evit': False,
  'bloquei': False,
  'carta': False,
  'senh': False,
  'refac': False,
  'operaca': False,
  'acess': False,
  'httpluxsmssantandercom5511981870490': False,
  'faç': False,
  'login': False,
  'aplic': False,
  'olymptrad': False,
  '170233180207': False,
  'br': False,
  'turv': False,
  'bb': False,
  'informavalid': False,
  'transtorn': False,
  'wwwbbrasildesbloqueiocomz0my2dd8': False,
  'inform': False,
  'j': False,
  'expir': False,
  'valid': False,
  'wwwbbrasildesbloqueiocomus16p1gq': False,
  'wwwbbrasildesbloqueiocomlgqrka95': Fal

In [73]:
tamanho_base_teste = len(frases_teste)
quantida_erros = len(erros)

porcentagem_erros = (quantida_erros * 100) / tamanho_base_teste
porcentagem_acertos = 100 - porcentagem_erros

print("O algoritmo classificou {:.4}% das frases corretamente".format(porcentagem_acertos))
print("O algoritmo classificou {:.4}% das frases incorretamente".format(porcentagem_erros))

O algoritmo classificou 98.42% das frases corretamente
O algoritmo classificou 1.583% das frases incorretamente


### 5.1.Matriz de confusão

Outra forma de visualização de erros e acertos é a construção da matriz de confusão:

- Importamos o pacote do ```nltk``` com a função da matriz de confusão.
- Criamos duas listas, uma com o resultado ```esperado``` e outra com o resultado ```previsto```, sendo que o esperado é o resultado desejado como resposta, e o previsto é de fato a classificação realizada.
- A saída do algoritmo mostra uma matriz com linhas que representam o esperado e colunas que representam o previsto.
- A diagonal principal indica a quantidade de acertos de cada classe.

Esses resultados apresentam as classes que o algoritmo está mais errando e/ou acertando. Assim, é possível tomar decisões para melhorar a implementação da base de dados, bem como alguns parâmetros de otimização do algoritmo.

In [74]:
from nltk.metrics import ConfusionMatrix

In [75]:
esperado = []
previsto = []
for (frase, classe) in frases_teste:
    resultado = classificador.classify(frase)
    previsto.append(resultado)
    esperado.append(classe)

matriz = ConfusionMatrix(esperado, previsto)
print(matriz)

        |   b     |
        |   l     |
        |   o     |
        |   c     |
        |   k     |
        |   e   o |
        |   d   k |
--------+---------+
blocked |<301> 12 |
     ok |   7<880>|
--------+---------+
(row = reference; col = test)



# 6.Revalidar o algoritmo de treino aplicando validação cruzada 

Como a acurácia foi bastante alta, precisamos revalidar o algoritmo de treino com validação cruzada para garantir que não há um viés específico da base de treino devido a separação aleatória realizada pela função train_test_split. Como a biblioteca NLTK não tem suporte direto para cross validation, vamos realizar o processo manualmente.

In [76]:
dados_kfold = carregar_dados(dados)

In [77]:
base_kfold = estruturar_dados(dados_kfold)

In [78]:
lista_acurácia = []
num_folds = 10
tamanho_fold = int(len(base_kfold)/num_folds)
for i in range(num_folds):
    teste = base_kfold[i*tamanho_fold:][:tamanho_fold]
    treino = base_kfold[:i*tamanho_fold] + base_kfold[(i+1)*tamanho_fold:]
    
    classificador = nltk.NaiveBayesClassifier.train(treino)
    acurácia = nltk.classify.accuracy(classificador, teste)*100
    print("A acurácia da classificação da base teste {} é de {:.4}%".format(i + 1, acurácia))
    lista_acurácia.append(acurácia)

A acurácia da classificação da base teste 1 é de 49.83%
A acurácia da classificação da base teste 2 é de 99.83%
A acurácia da classificação da base teste 3 é de 81.0%
A acurácia da classificação da base teste 4 é de 99.5%
A acurácia da classificação da base teste 5 é de 83.83%
A acurácia da classificação da base teste 6 é de 96.5%
A acurácia da classificação da base teste 7 é de 93.83%
A acurácia da classificação da base teste 8 é de 94.0%
A acurácia da classificação da base teste 9 é de 97.0%
A acurácia da classificação da base teste 10 é de 98.17%


In [79]:
import numpy as np

In [80]:
np.array(lista_acurácia).mean()

89.35

A acurácia média do teste com validação cruzada K-Fold (k=10) foi de 89,35%, o que significa que não temos problemas de overfitting e podemos agora realizar o treino com a base completa.

# 6.Treino com a base completa

Como o modelo foi validado, podemos agora treiná-lo com a base de treino original completa.

In [81]:
base_treino = carregar_dados(dados)

In [82]:
len(base_treino)

6000

In [83]:
base_final = estruturar_dados(base_treino)

In [84]:
%%time
# Cria o classificador (tabela de probabilidade) com base no conjunto de treinamento
classificador = nltk.NaiveBayesClassifier.train(base_final)

Wall time: 1min 30s


In [85]:
# Retorna as classes da base de dados (ok, blocked)
print(classificador.labels())

['blocked', 'ok']


In [86]:
# Retorna os 10 atributos mais significativos
print(classificador.show_most_informative_features(10))

Most Informative Features
                      bb = True           blocke : ok     =   1778.2 : 1.0
                     fot = True           blocke : ok     =    396.8 : 1.0
                      br = True               ok : blocke =    314.7 : 1.0
                       j = True           blocke : ok     =    266.9 : 1.0
                   debit = True               ok : blocke =    208.2 : 1.0
                 prezado = True           blocke : ok     =    192.9 : 1.0
                   local = True           blocke : ok     =    166.9 : 1.0
                   login = True               ok : blocke =    121.9 : 1.0
                   aplic = True               ok : blocke =    104.2 : 1.0
                     lig = True               ok : blocke =    100.0 : 1.0
None


# 7.Salvar o modelo criado para aplicar em novos dados

In [87]:
import pickle

In [88]:
def salvar_modelo(modelo, nome_arquivo):
    nome = str(nome_arquivo) + ".pickle"
    try:
        salvar_modelo = open(nome,"wb")
        pickle.dump(modelo, salvar_modelo)
        salvar_modelo.close()
        return True
    except Exception as e:
        return e

In [89]:
if salvar_modelo(classificador,'naivebayes'):
    print("Modelo salvo para ser utilizado no futuro")

Modelo salvo para ser utilizado no futuro


In [90]:
print("Tempo total para executar esse notebook foi de {} segundos".format(time() - ti))

Tempo total para executar esse notebook foi de 1202.9029409885406 segundos
